# Philips Supplier Sustainability Analytics - Starter Notebook

This notebook provides a minimal starting point for the assignment.

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Load Data

In [18]:
# Load the data
df = pd.read_excel('data/SSP_Data.xlsx')

# Display basic information
print(f"Dataset shape: {df.shape}")
print(f"Number of unique suppliers: {df['ID'].nunique()}")
df.head()

Dataset shape: (1262, 495)
Number of unique suppliers: 463


,Index,ID,Sequence,Assessment Year,Assessment type,Country,Val_Score,Val_Environment,Val_Health and Safety,Val_Business Ethics,...,Q953_3,Q953_4,Q953_5,Q1443,Q1446,Q1449,Q1450,Q954,Q956,Q1452
0,256-1,256,1,2017,DV - Desktop Validation,China,0.67175,0.7110,0.7614,0.694,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,256-2,256,2,2018,SA - Site Assessment,China,0.73240,0.8800,0.7210,0.721,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,256-3,256,3,2019,SA - Site Assessment,China,0.71215,0.8250,0.6536,0.721,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,256-4,256,4,2020,DV - Desktop Validation,China,0.76200,0.8550,0.7140,0.800,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,745-1,745,1,2019,SA - Site Assessment,Indonesia,0.56180,0.8336,0.4546,0.355,...,1.0,0.0,0.0,0.0,NaN,NaN,0.0,1.0,1.0,1.0


## 3. Your Analysis Here

**Reminder:** Split by supplier ID, not randomly, to avoid data leakage.

In [19]:
df_ohe = pd.get_dummies(df, columns=['Country', 'Assessment type'])

In [16]:
df_ohe = df_ohe.dropna(axis=1, how="any")

In [26]:
completeness = 1 - df_ohe.isna().mean()

In [27]:
to_drop = completeness[completeness < 0.80].index

In [28]:
df_ohe = df_ohe.drop(columns=to_drop)

In [29]:
df_ohe.describe()

,ID,Sequence,Assessment Year,Val_Score,Val_Environment,Val_Health and Safety,Val_Business Ethics,Val_Labor and Human Rights,SAQ_Score,SAQ_Environment,...,Q1434,Q1441,Q953_1,Q953_2,Q953_3,Q953_4,Q953_5,Q1443,Q1450,Q954
count,1262.000000,1262.000000,1262.000000,1262.000000,1262.000000,1262.000000,1262.000000,1262.000000,1262.000000,1262.000000,...,1205.000000,1226.000000,1137.000000,1134.000000,1133.000000,1132.000000,1128.000000,1197.000000,1182.000000,1181.000000
mean,369.194929,2.470681,2020.011886,0.479522,0.571900,0.493273,0.401786,0.462120,0.594701,0.673371,...,0.790041,0.915987,0.889182,0.880071,0.876434,0.871025,0.012411,0.732665,0.753807,0.704488
std,222.489020,1.470410,2.544890,0.202770,0.257746,0.233253,0.235719,0.207747,0.176915,0.225710,...,0.407448,0.277521,0.314045,0.325022,0.329231,0.335321,0.110762,0.442754,0.430975,0.456466
min,1.000000,1.000000,2016.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,180.250000,1.000000,2018.000000,0.343036,0.402970,0.352250,0.217000,0.326075,0.516556,0.585200,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,349.500000,2.000000,2019.000000,0.501121,0.628500,0.526900,0.398500,0.484200,0.642925,0.763000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000
75%,553.000000,3.000000,2022.000000,0.639477,0.780200,0.669430,0.588250,0.617452,0.713575,0.825000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000
max,783.000000,7.000000,2025.000000,0.895400,0.987400,0.960000,0.940000,0.903400,0.970775,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
